In [49]:
import requests
from weakref import ref as _wref
from collections import OrderedDict as OD
from copy import deepcopy, copy
import shutil

from ruamel.yaml import YAML, RoundTripDumper, RoundTripLoader
yaml = YAML(typ='safe')   # default, if not specfied, is 'rt' (round-trip)

def yload(fp):
    if isinstance(fp, dict):
        return fp  # yaml already parsed
    if isinstance(fp, str):
        fp = open(fp, 'r')
    try:
        rv = yaml.load(fp)
    except Exception:
        fp.close()
        raise
    else:
        fp.close()
    return rv

In [50]:
class HasCTXBase():
    def __init__(self, ctx):
        self._ctx = _wref(ctx)
    @property 
    def ctx(self):
        return self._ctx()

In [51]:
def eprint(lines, msg):
    lines.append(msg)
    print(msg)

def extra_schema(remaining, original):
    lines = [""]
    for key in remaining:
        eprint(lines, "REMAINING: %s=%s"%(key, repr(remaining[key])))
    for key, value in original.items():
        eprint(lines, "HAD: %s=%s"%(key, repr(value)))
    return "\n".join(lines)

class SchemaBase(HasCTXBase):
    def __init__(self, ctx):
        super().__init__(ctx)
        self._path = None
        self._anon_name = None
        
    def setAnonName(self, name):
        self._anon_name = name
        
    def setReferencePath(self, path):
        self._path = path
    
    def getReferencepath(self):
        return self._path
    
    def isAnonymous(self):
        return self._path is None
        
    def get_FileName(self):
        return self.name + ".py"
        
    def get_InheritInfo(self):
        raise NotImplementedError
        
    def get_InitString(self):
        raise NotImplementedError
        
    def get_AttributeList(self):
        raise NotImplementedError
        
    def get_Attributes(self):
        raise NotImplementedError
        
    def get_SuperImportString(self):
        raise NotImplementedError("not implemented for type %s"%self.__class__.__name__)
        
    def get_AsImport(self):
        return (self.get_FileName()[:-3], self.get_ClassName())
    
    def get_ClassName(self):
        if self.isAnonymous():
            if self._anon_name is None:
                return self.get_TypeName()
            else:
                return self._anon_name
        else:
            return self.name
        
    def get_IsNativeType(self):
        if isinstance(self, BasicSchema):
            return True
        if isinstance(self, EnumSchema) and isinstance(self.value, BasicSchema):
            return True
        return False
        
    def get_AnonTypes(self):
        return []
    
    def get_TypeName(self):
        return self.__class__.__name__
    
    def _get_AllAnons(self, obj, anons):
        for o in obj.get_AnonTypes():
            self._get_AllAnons(o, anons)
            if o.isAnonymous():
                anons.append(o)
                
    def get_Anons(self):
        anons = []
        self._get_AllAnons(self, anons)
        return anons
    
    def get_SuperName(self):
        return ""
    
    def get_GlobalImports(self):
        return []
    
    def get_AsAssignment(self):
        return self.get_ClassName() + ".getDefault()"


class StringSchema():
    _attrib = {"default", "nullable", "readOnly", "maxLength", "minLength", "format"}
    
class IntSchema():
    _attrib = {"default", "minimum", "maximum", "format", "nullable"}
    
class BoolSchema():
    _attrib = {"default", "nullable"}
    
class NumberSchema():
    _attrib = {"default", "minimum", "maximum", "format", "nullable", "readOnly"}
    
class DateTimeSchema():
    _attrib = {"default", "nullable", "format"}
    
class DateSchema():
    _attrib = {"default", "nullable", "format"}

class BasicSchema(SchemaBase):
    _basic_types = {
        "string": StringSchema,
        "integer": IntSchema,
        "boolean": BoolSchema,
        "number": NumberSchema,
        "date-time": DateTimeSchema,
        "date": DateSchema
    }
    def __init__(self, ctx, name, theschema):
        super().__init__(ctx)
        self.name = name
        
        schema = copy(theschema)
        self.type = schema.pop('type')
        self.description = schema.pop('description', None)
        
        attrib = {
            "default": None,
            "required": False,
            "minimum": None,
            "maximum": None,
            "format": None,
            "nullable": None,
            "readOnly": False,
            "maxLength": None,
            "minLength": None,
        }
        
        if self.type not in self._basic_types:
            raise ValueError(self.type)
        
        self.attrib = {}
        allowed = self._basic_types[self.type]._attrib
        for key, default in attrib.items():
            if key in allowed:
                self.attrib[key] = schema.pop(key, default)
        
        schema.pop('example', None)
            
        assert not schema, extra_schema(schema, theschema)
        
    def get_Default(self):
        return repr(self.attrib.get('required', None))
        
    def get_Type(self):
        return self._basic_types[self.type]
    
    def get_SuperName(self):
        return self.get_Type().__name__
    
    def get_FieldType(self):
        if self.type == "string":
            return "str"
        elif self.type == "integer":
            return "int"
        elif self.type == "boolean":
            return "bool"
        elif self.type == "number":
            return "float"
        elif self.type == "date-time":
            return "datetime"
        elif self.type == "date":
            return "date"
        else:
            raise TypeError(self.type)
    
    def get_InheritInfo(self):
        # this recreates the schema used to create the 
        # BasicSchema object, minus the descr
        info = {"type": self.type}
        info.update(self.attrib)
        return info
    
    def get_InitString(self):
        return "self, value=%s, **kw"%repr(self.attrib.get("default", None))
    
    def get_AttributeList(self):
        return list(self.attrib)
    
    def get_Attributes(self):
        return self.attrib.items()
    
    def get_SuperImportString(self):
        return "schemabase", self.get_SuperName()
    
    def get_LocalImports(self):
        return [self.get_SuperImportString()]
    
    
class DateTimeBasicSchema(BasicSchema):
    def __init__(self, ctx, name, theschema):
        schema = copy(theschema)
        schema['type'] = "date-time"
        super().__init__(ctx, name, schema)
        
    def get_GlobalImports(self):
        return [('datetime', 'datetime')]
        
        
class DateBasicSchema(BasicSchema):
    def __init__(self, ctx, name, theschema):
        schema = copy(theschema)
        schema['type'] = "date"
        super().__init__(ctx, name, schema)
        
    def get_GlobalImports(self):
        return [('datetime', 'date')]
        
        
class EnumSchema(SchemaBase):
    def __init__(self, ctx, name, theschema):
        schema = copy(theschema)
        self.allowed = set(schema.pop('enum'))
        self.name = name
        super().__init__(ctx)
        self.value = ctx.parse_a_schema("value", schema)
        self.description = schema.pop("description", self.value.description)
    
    def get_InheritInfo(self):
        info = {
            "allowed": self.allowed,
            "value": self.value
        }
        return info
    
    def get_LocalImports(self):
        if self.value.isAnonymous():
            return self.value.get_LocalImports()
        return [self.value.get_AsImport()]
    
    def get_FieldType(self):
        return self.value.get_FieldType()
    
    def get_AnonTypes(self):
        if self.value.isAnonymous():
            return [self.value]
        return []
    
    def get_Default(self):
        return self.value.get_Default()
    
    def get_SuperImportString(self):
        return self.value.get_SuperImportString()
    
    def get_GlobalImports(self):
        if self.value.isAnonymous():
            return self.value.get_GlobalImports()
        
        
class ArraySchema(SchemaBase):
    def __init__(self, ctx, name, theschema):
        super().__init__(ctx)
        
        schema = copy(theschema)
        self.name = name
        
        items = copy(schema.pop('items'))  # don't forget copy!
        attrib = {
            "minItems": None,
            "uniqueItems": None
        }
        
        # these attrib can be property of the array, or property
        # of the item in the array. 
        # I didn't make this up. 
        self.attrib = OD()
        for k, v in attrib.items():
            v = schema.pop(k, v)
            v = items.pop(k, v)
            self.attrib[k] = v
        
        self.type = ctx.parse_a_schema("items", items)
        self.description = schema.pop('description', None)
        
        schema.pop('example', None)
        schema.pop('explode', None)
        schema.pop('type')
        assert not schema, extra_schema(schema, theschema)
    
    def get_InheritInfo(self):
        info = {
            "type": self.type,
            "attrib": copy(self.attrib),
        }
        return info
    
    def get_LocalImports(self):
        if self.type.isAnonymous():
            imports = set(self.type.get_LocalImports())
        else:
            imports = {self.type.get_AsImport()}
        imports.add(("schemabase", "ArraySchema"))
        return imports
    
    def get_SuperName(self):
        return "ArraySchema"
    
    def get_GlobalImports(self):
        if self.type.isAnonymous():
            return self.type.get_GlobalImports()
        return []
    
    def get_FieldType(self):
        return self.get_ClassName()
        
    def get_AnonTypes(self):
        if self.type.isAnonymous():
            return [self.type]
        return []
    
    def get_Default(self):
        return self.type.get_Default()
    
    def get_AsAssignment(self):
        return self.get_ClassName() + "()"
        
    

class ObjectSchema(SchemaBase):
    def __init__(self, ctx, name, theschema):
        super().__init__(ctx)
        
        schema = copy(theschema)
        self.name = name
        self.is_empty = False
        
        self.description = schema.pop('description', None)
        self.required = schema.pop("required", [])
        self.nullable = schema.pop('nullable', False)
        
        self.fields = OD()
        for k, v in schema.pop('properties', {}).items():
            self.fields[k] = ctx.parse_a_schema(k, v)
            
        schema.pop('example', None)
            
        if not self.fields:
            self.is_empty = True
        # it is possible for some of the objects to 
        # be of type "object" and otherwise empty, with no
        # properties. This is fine, it means the return object
        # will be a JSON blob of whatever (??)
        
    def get_InitString(self):
        args = ["self"]
        for name, _ in self.get_Fields():
            if name == "self":
                name = "_self"
            args.append("%s=None"%name)
        args.append("**kw")
        return ", ".join(args)
        
    def get_AnonTypes(self):
        return [v for v in self.fields.values() if v.isAnonymous()]
        
    def get_Fields(self):
        ret = []
        for k, v in self.fields.items():
            if k == 'self':
                k = '_self'
            ret.append((k,v))
        return ret
            
    def get_InheritInfo(self):
        if self.is_empty:
            # this shouldn't happen but whatever
            return None
        
        info = {
            "required": self.required,
            "nullable": self.nullable,
        }
        
        info["fields"] = self.fields
        return info
    
    def get_LocalImports(self):
        strings = set()
        for f in self.fields.values():
            if not f.isAnonymous():
                strings.add(f.get_AsImport())
            else:
                strings.update(f.get_LocalImports())
        return strings
    
    def get_GlobalImports(self):
        imports = set()
        for f in self.fields.values():
            if f.isAnonymous():
                imports.update(f.get_GlobalImports())
        return list(imports)
    
    def get_Default(self):
        return self.get_AsAssignment()
    
    def get_FieldType(self):
        return self.get_ClassName()    
    
    
class OneOfSchema(SchemaBase):
    def __init__(self, ctx, name, theschema):
        super().__init__(ctx)
        
        schema = copy(theschema)
        self.name = name
        self.union = []
        for obj in schema.pop('oneOf'):
            sch = ctx.parse_a_schema("value", obj)
            self.union.append(sch)
            
        self.discriminator = schema.pop("discriminator")
        self.discriminator_mapping = {}
        for k, v in self.discriminator['mapping'].items():
            self.discriminator_mapping[k] = ctx.resolve_schema(v)
            
        self.description = schema.pop("description", None)
        
    def get_UnionMapping(self):
        return self.discriminator_mapping.items()
            
    def get_LocalImports(self):
        strings = set()
        for obj in self.union:
            if obj.isAnonymous():
                strings.update(obj.get_LocalImports())
            else:
                strings.add(obj.get_AsImport())
            
        return strings
    
    def get_FieldType(self):
        return self.get_ClassName()
    
    def get_AnonTypes(self):
        return [o for o in self.union if o.isAnonymous()]
    
    def get_Default(self):
        return self.get_ClassName() + "()"

In [52]:
class Context():
    def __init__(self, y):
        self.schemas = OD()
        self.paths = OD()
        self.parameters = OD()
        self.responses = OD()
        self.request_bodies = OD()
        self.classes = OD()
        self.yaml = y
        
    def convert_paths_to_api(self):
        for path in self.paths.values():
            for m in path.methods.values():
                assert len(m.tags) == 1
                name = m.tags[0].title().replace(" ", "")
                if name in self.classes:
                    cls = self.classes[name]
                else:
                    cls = ClassProxy(name)
                    self.classes[name] = cls
                cls.methods.append(m)
    
    def _get_anons(self, obj, anons):
        for o in obj.get_AnonTypes():
            anons.append(o)
            self._get_anons(o, anons)
                  
    def assign_anonymous_names(self):
        # build full list of all schemas
        # including anons in paths, responses, params..
        # TODO: others
        anons = []
        for sch in self.schemas.values():
            self._get_anons(sch, anons)
            
        for param in self.parameters.values():
            self._get_anons(param, anons)
            
        for rsp in self.responses.values():
            self._get_anons(rsp, anons)
            
        for path in self.paths.values():
            self._get_anons(path, anons)

        anons = set(anons)
            
        amap_counter = {}
        for a in anons:
            key = a.name
            if key not in amap_counter:
                amap_counter[key] = 0
            amap_counter[key] += 1
            n = amap_counter[key]
            aname = key + "_AX" + str(n)
            a.setAnonName(aname)        
        
    def resolve_schema(self, path):
        if path in self.schemas:
            return self.schemas[path]
        
        # resolve if not found
        parts = path.split("/")
        if parts[0] != "#":
            raise ValueError(path)
        ob = self.yaml
        for p in parts[1:]:
            ob = ob[p]
        
        sch = self._parse_a_schema(path, p, ob)
        self.schemas[path] = sch
        sch.setReferencePath(path)
        return sch
    
    def _assert_combo(self, schema):
        # verifies invariants assumed about 
        # the allOf/oneOf/anyOf types
        found = 0
        scopy = schema.copy()
        hkey = None
        for key in "allOf", "anyOf", "oneOf":
            if key in schema: 
                found += 1
                scopy.pop(key)
                hkey = key
        if found != 1:
            raise ValueError(schema)
        scopy.pop("description", None)
        scopy.pop("example", None)
        if hkey != "allOf":
            scopy.pop("discriminator", None)
        assert not scopy, extra_schema(scopy, schema)
        
            
    def parse_a_schema(self, name, schema):
        if isinstance(schema, SchemaBase):
            return schema
        return self._parse_a_schema(None, name, schema)
    
    def _parse_a_schema(self, path, name, schema):
        if "properties" in schema:
            return self._parse_object_schema(path, name, schema)
        elif "items" in schema:
            assert schema['type'] == 'array'
            return self._parse_array_schema(path, name, schema)
        elif "allOf" in schema:
            self._assert_combo(schema)
            return self._parse_allof_schema(path, name, schema)
        elif "oneOf" in schema:
            self._assert_combo(schema)
            return self._parse_union_schema(path, name, schema)
        elif "anyOf" in schema:
            self._assert_combo(schema)
            return self._parse_freelove_schema(path, name, schema)
        elif '$ref' in schema:
            return self._parse_ref_schema(path, name, schema)
        elif 'type' in schema:
            if 'enum' in schema:
                return self._parse_enum_schema(path, name, schema)
            elif schema['type'] == 'object':
                return self._parse_object_schema(path, name, schema)
            else:
                return self._parse_basic_schema(path, name, schema)
        else:
            raise ValueError(schema)
            
    def _parse_object_schema(self, path, name, schema):
        return ObjectSchema(self, name, schema)
    
    def _parse_array_schema(self, path, name, schema):
        return ArraySchema(self, name, schema)
    
    def _iter_allof(self, schema, name):
        s = schema['allOf']
        if isinstance(s, list):
            for d in s:
                yield from d.items()
        elif isinstance(s, dict):
            yield from s.items()
    
    def _parse_allof_schema(self, path, name, schema):
        # an "allOf" schema is really less of a distinct type
        # and really more of an object that inherits the types
        # of its "all of" members
        
        fix_allof(schema, name)
        
        new_schema = OD()
        parents = []
        for k, v in self._iter_allof(schema, name):
            if k == "$ref":
                parent = self.resolve_schema(v)
                parents.append(parent)
            else:
                new_schema[k] = v
        
        # it is possible to "inherit" from a primitive type, the same way
        # that python allows you to:
        #
        # class MyInteger(int):
        #     pass
        
        if len(parents) == 1 and "properties" not in new_schema:
            p = parents[0]
            if isinstance(p, BasicSchema):
                info = p.get_InheritInfo()
                info['description'] = new_schema.get('description', None)
                sch = BasicSchema(self, name, info)
            
            # same with arrays, but there's a problem because the schema
            # for arrays is incompatible with converting back into 
            # a dict...
            elif isinstance(p, ArraySchema):
                info = p.get_InheritInfo()
                sch = ArraySchema.__new__(ArraySchema)
                super(ArraySchema, sch).__init__(self)
                sch.name = name
                sch.type = info['type']
                sch.description = new_schema.get('description', None)
                sch.attrib = info['attrib']
            
            # enums could be done the same as basic properties, but it is easier
            # to just force the values the same way as is done for array.
            elif isinstance(p, EnumSchema):
                info = p.get_InheritInfo()
                sch = EnumSchema.__new__(EnumSchema)
                super(EnumSchema, sch).__init__(self)
                sch.name = name
                sch.allowed = info['allowed']
                sch.value = info['value']
            
            elif (isinstance(p, ObjectSchema)):
                sch = self._inherit_obj_schema(name, new_schema, parents)
            
            else:
                raise ValueError("%s %s" % (type(p), name))
            return sch
        
        # must be complex object
        return self._inherit_obj_schema(name, new_schema, parents)
    
    def _inherit_obj_schema(self, name, new_schema, parents):
        sch = ObjectSchema(self, name, new_schema)
        for p in parents: 
            info = p.get_InheritInfo()
            for k, v in info['fields'].items():
                if k in sch.fields:
                    raise ValueError("Duplicate value for allOf inheritance: '%s' for '%s'"%(k, name))
                sch.fields[k] = v
        return sch
    
    def _parse_union_schema(self, path, name, schema):
        return OneOfSchema(self, name, schema)
    
    def _parse_freelove_schema(self, path, name, schema):
        print("anyOf:", path)
        raise NotImplementedError
    
    def _parse_ref_schema(self, path, name, schema):
        path = schema['$ref']
        
        if path in self.schemas:
            return self.schemas[path]
        
        return self.resolve_schema(path)
        
#         assert len(schema) == 1
#         sch = self.resolve_schema(path)
#         if sch is not None:
#             return ProxySchema(ctx, name, sch)
    
    def _parse_basic_schema(self, path, name, schema):
        format = schema.get("format", None)
        if format == "date-time":
            return DateTimeBasicSchema(self, name, schema)
        elif format == "date":
            return DateBasicSchema(self, name, schema)
        else:
            return BasicSchema(self, name, schema)
    
    def _parse_enum_schema(self, path, name, schema):
        return EnumSchema(self, name, schema)
    
    def parse_a_parameter(self, data):
        if isinstance(data, Parameter):
            return data
        return self._parse_a_parameter(data)
    
    def _parse_a_parameter(self, data):
        if "$ref" in data:
            return self.resolve_param(data['$ref'])
        else:
            print(repr(data))
            raise ValueError(data)
    
    def resolve_param(self, path):
        if path in self.parameters:
            return self.parameters[path]
        
        # resolve if not found
        parts = path.split("/")
        if parts[0] != "#":
            raise ValueError(path)
        ob = self.yaml
        for p in parts[1:]:
            ob = ob[p]
        
        param = Parameter(self, p, ob)
        self.parameters[path] = param
        param.setReferencePath(path)
        return param
    
    def resolve_reqbody(self, path):
        if path in self.request_bodies:
            return self.request_bodies[path]
        
        # resolve if not found
        # paths are a bit special
        parts = path.split("/")
        if parts[0] != "#":
            raise ValueError(path)
        ob = self.yaml
        for p in parts[1:]:
            ob = ob[p]
            
        rsp = RequestBody(self, p, ob)
        self.request_bodies[path] = rsp
        rsp.setReferencePath(path)
        return rsp
    
    def resolve_response(self, path):
        if path in self.responses:
            return self.responses[path]
        
        # resolve if not found
        # paths are a bit special
        parts = path.split("/")
        if parts[0] != "#":
            raise ValueError(path)
        ob = self.yaml
        for p in parts[1:]:
            ob = ob[p]
            
        rsp = Response(self, p, ob)
        self.responses[path] = rsp
        rsp.setReferencePath(path)
        return rsp
    
    def resolve_path(self, path):
        if path in self.paths:
            return self.paths[path]
            
        ob = self.yaml['paths'][path]
        pob = Path(self, path, ob)
        self.paths[path] = pob
        return pob

In [53]:
class Parameter(HasCTXBase):
    def __init__(self, ctx, name, theparam):
        super().__init__(ctx)
        
        param = copy(theparam)
        
        self.name = name
        self.parameter_name = param.pop('name', name)
        self.schema = ctx.parse_a_schema(name, param.pop("schema"))
        self.loc = param.pop("in")
        self.description = param.pop('description', None)
        self.required = param.pop('required', False)
        self.explode = param.pop('explode', None)
        param.pop("example", None)
        assert not param, extra_schema(param, theparam)
        
        self._path = None
        
    def setReferencePath(self, path):
        self._path = path
        
    def get_AnonTypes(self):
        if self.schema.isAnonymous():
            return [self.schema]
        return []
    
    
class Header():
    _allowed_types = {
        'string',
        'integer',
        'number',
        'date-time',
        'date'
    }
    def __init__(self, key, theheader):
        self.type = theheader['schema']['type']
        if self.type not in self._allowed_types:
            raise TypeError(self.type)
        self.key = key
        self.description = theheader.get('description', None)
        
    
class BodyObject(HasCTXBase):
    def __init__(self, ctx, name, thersp):
        super().__init__(ctx)
        
        rsp = copy(thersp)
        
        self.name = name
        rsp.pop("example", None)
        content = rsp.pop('content', None)
        self.description = rsp.pop('description', None)
        if content is None:
            self.content_type = None
            self.schema = None
        else:
            assert len(content) == 1
            self.content_type, data = next(iter(content.items()))
            self.schema = ctx.parse_a_schema("_"+self.name+"_Rsp", data['schema'])
        
        self._path = None
        
    def setReferencePath(self, path):
        self._path = path
        
    def isAnonymous(self):
        return self._path is None
        
    def get_AnonTypes(self):
        if self.schema and self.schema.isAnonymous():
            return [self.schema]
        return []
    
    def get_SchemaImports(self):
        if self.schema:
            if self.schema.isAnonymous():
                return self.schema.get_LocalImports()
            else:
                return [self.schema.get_AsImport()]
        return []
    
    def get_GlobalImports(self):
        if self.schema:
            return self.schema.get_GlobalImports()
        return set()
        
    def _get_AllAnons(self, obj, anons):
        for o in obj.get_AnonTypes():
            self._get_AllAnons(o, anons)
            anons.append(o)
                
    def get_Anons(self):
        anons = []
        self._get_AllAnons(self, anons)
        return anons
    
    def get_ClassName(self):
        return self.name
    
    def get_FileName(self):
        return self.get_ClassName() + ".py"
    
    def get_HasSchema(self):
        return self.schema is not None
    
    def get_Description(self):
        return repr(self.description)
    
    def get_AsImport(self):
        return self.get_ClassName(), self.get_ClassName()
    
    
class RequestBody(BodyObject):
    def get_SerializationMethod(self):
        if self.content_type == 'application/json':
            return "to_json"

    
class Response(BodyObject):
    def __init__(self, ctx, name, thersp):
        super().__init__(ctx, name, thersp)        
        
        self.headers = []
        for header, val in thersp.get('headers', {}).items():
            self.headers.append(Header(header, val))

In [54]:
class Method(HasCTXBase):
    def __init__(self, ctx, pathob, method, mdata):
        super().__init__(ctx)
        self._path = None
        self.path = pathob
        
        data = copy(mdata)
        self.method = method
        self.summary = data.pop('summary', None)
        self.description = data.pop('description', None)
        self.name = data.pop('operationId')
        self.tags = data.pop('tags')
        self.security = data.pop('security', None)  # I don't know what this does
        
        self.parameters = []
        for p in data.pop('parameters', []):
            self.parameters.append(ctx.parse_a_parameter(p))
            
        self._param_map = {p.name: p for p in self.parameters}
            
        self.responses = {}
        for code, rdata in data.pop('responses', {}).items():
            if "$ref" in rdata:
                rspob = ctx.resolve_response(rdata["$ref"])
            else:
                rspob = Response(ctx, self.name+"_"+str(code)+"_Rsp", rdata)
            self.responses[code] = rspob
            
        request_body = mdata.pop('requestBody', None)
        if request_body:
            if len(request_body) == 1 and "$ref" in request_body:
                self.request_body = ctx.resolve_reqbody(request_body['$ref'])
            else:
                self.request_body = RequestBody(ctx, self.name+"_RequestBody", request_body)
        else:
            self.request_body = None
            
    def get_HasBody(self):
        return self.request_body is not None
            
    def get_PathFormatArgsAsString(self):
        args = []
        for p in self.parameters:
            if p.loc == "path":
                args.append("%s=%s"%(p.parameter_name, p.parameter_name))
        return ", ".join(args)
    
    def get_QueryParams(self):
        return self._get_sorted_params()[1]
        
    @property
    def path(self):
        return self._path()
    
    def get_MethodType(self):
        return self.method
    
    def get_Responses(self):
        r = []
        for k, v in self.responses.items():
            if k == "default":
                r.append(('"%s"'%k,v))
            else:
                r.append((int(k), v))
        return sorted(r, key=lambda kv: kv[0] if isinstance(kv[0], int) else 999999999999999)
    
    @path.setter
    def path(self, value):
        self._path = _wref(value)
        
    def get_PathURL(self):
        return self.path.path
        
    def get_FunctionName(self):
        return self.name
    
    def _pathidx(self, path, key):
        try:
            return path.index(key)
        except ValueError:  # not found
            return None
    
#     def _get_sorted_params(self):
#         path = self.path.path
        
#         def _remaining(params):
#             return [p for p in self.parameters if p not in params]
        
#         params = sorted((p for p in self.parameters if "{%s}"%p.name in path), key=lambda p: path.index("{%s}"%p.name))
        
# #         # required, no defaults
# #         for p in _remaining(params):
# #             if p.required and p.schema.default is None:
# #                 params.append(p)
        
#         # required
#         for p in _remaining(params):
#             if p.required:
#                 params.append(p)
                
#         for p in _remaining(params):
#             params.append(p)
            
#         return params

    def _get_sorted_params(self):
        path = self.path.path
        in_path = sorted((p for p in self.parameters if "{%s}" % p.parameter_name in path), 
                         key=lambda p: path.index("{%s}" % p.parameter_name))
        in_query = [p for p in self.parameters if p.loc == 'query']
        if len(in_path) + len(in_query) != len(self.parameters):
            for p in self.parameters:
                if p not in in_path and p not in in_query:
                    print("? param '%s': '%s', '%s'" % (self.name, p.parameter_name, p._path))
            raise ValueError(self.parameters)
        return in_path, in_query
    
    def get_FuncParamString(self):
        path, query = self._get_sorted_params()
        args = ["self"]
        for p in path:
            args.append(p.parameter_name)
        if self.request_body:
            args.append("body=None")
        for p in query:
            args.append("%s=NoArg"%p.parameter_name)
        return ", ".join(args)
    
    def get_GlobalImports(self):
        imports = set()
        for p in self.parameters:
            imports.update(p.schema.get_GlobalImports())
        for rsp in self.responses.values():
            imports.update(rsp.get_GlobalImports())
        return imports
    
    def get_SchemaImports(self):
        imports = set()
        for p in self.parameters:
            if p.schema.isAnonymous():
                imports.update(p.schema.get_LocalImports())
            else:
                imports.add(p.schema.get_AsImport())
        for rsp in self.responses.values():
            if rsp.isAnonymous():
                imports.update(rsp.get_SchemaImports())
        if self.request_body:
            if self.request_body.schema.isAnonymous():
                imports.update(self.request_body.schema.get_LocalImports())
            else:
                imports.add(self.request_body.schema.get_AsImport())
        return imports
                
    def get_ResponseImports(self):
        imports = set()
        for p in self.parameters:
            for rsp in self.responses.values():
                if not rsp.isAnonymous():
                    imports.add(rsp.get_AsImport())
        return imports
    
    def get_AnonTypes(self):
        anons = []
#         for p in self.parameters:
#             anons.extend(p.get_AnonTypes())
        for rsp in self.responses.values():
            anons.extend(rsp.get_AnonTypes())
        if self.request_body:
            anons.extend(self.request_body.get_AnonTypes())
        return anons
                

class Path(HasCTXBase):
    
    _methodtypes = {
        "get",
        "post",
        "put",
        "delete"
    }
    
    def __init__(self, ctx, path, thedata):
        super().__init__(ctx)
        
        data = copy(thedata)
        self.path = path
        methods = {}
        while data:
            m, mdata = data.popitem()
            m = m.lower()
            if m in self._methodtypes:
                methods[m] = Method(ctx, self, m, mdata)
        self.methods = methods
    
    def get_AnonTypes(self):
        anons = []
        for m in self.methods.values():
            anons.extend(m.get_AnonTypes())
        return anons

In [55]:
class ClassProxy():
    def __init__(self, clsn):
        self.name = clsn
        self.methods = []
        
    def get_FileName(self):
        return self.name + ".py"
    
    def get_ClassName(self):
        return self.name
    
    def _get_anons(self, obj, anons, seen):
        for o in obj.get_AnonTypes():
            if o in seen:
                continue
            self._get_anons(o, anons, seen)
            anons.append(o)
            seen.add(o)
    
    def get_AnonSchemas(self):
        anons = []
        seen = set()
        for m in self.methods:
#             for p in m.parameters:
#                 self._get_anons(p, anons, seen)
            for rsp in m.responses.values():
                if rsp.isAnonymous():
                    self._get_anons(rsp, anons, seen)
            if m.request_body:
                self._get_anons(m.request_body, anons, seen)
        return anons
    
    def get_MethodList(self):
        return self.methods
    
    def get_SchemaImports(self):
        imports = set()
        for m in self.methods:
            imports.update(m.get_SchemaImports())
        return imports
        
    def get_ResponseImports(self):
        imports = set()
        for m in self.methods:
            imports.update(m.get_ResponseImports())
        return imports
    
    def get_GlobalImports(self):
        imports = set()
        for m in self.methods:
            imports.update(m.get_GlobalImports())
        return imports
    
    def get_AnonResponses(self):
        anons = set()
        for m in self.methods:
            for rsp in m.responses.values():
                if rsp.isAnonymous():
                    anons.add(rsp)
        return anons

In [56]:
def parse_yaml(y):
    global ctx
    ctx = Context(y)
    for k,v in y['components']['schemas'].items():
        ctx.resolve_schema("#/components/schemas/%s"%k)
        
    for k,v in y['components']['parameters'].items():
        ctx.resolve_param("#/components/parameters/%s"%k)
        
    for k, v in y['components']['responses'].items():
        ctx.resolve_response("#/components/responses/%s"%k)
        
    for k, v in y['paths'].items():
        ctx.resolve_path(k)
        
    ctx.assign_anonymous_names()
    
    ctx.convert_paths_to_api()
    
    return ctx


def fix_allof(schema, name):
    new = []
    for obj in schema['allOf']:
        if len(obj) > 1:
            for k, v in obj.items():
                new.append({k: v})
        else:
            new.append(obj)
    schema['allOf'] = new

In [57]:
try:
    yref
except NameError:
    thefile = "c:/users/nathan/downloads/helixalmrestapi.yaml"
    yref = yload(thefile)
y = deepcopy(yref)

In [58]:
def fix_shitty_yaml(y):
    yc = deepcopy(y['components']['schemas'])
    for k, v in yc.items():
        typ = 'allOf'
        if typ in v:
            v = v[typ]
            c = v.copy()
            new = []
            for ob in c:
                if len(ob) > 1:
                    for k2, v2 in ob.items():
                        new.append({k2: v2})
                else:
                    new.append(ob)
            y['components']['schemas'][k][typ] = new

#fix_shitty_yaml(y)
ctx = parse_yaml(y)

In [59]:
# for sch in ctx.schemas.values():
#     if sch is not None:
#         print(sch.name, ": ", sch.get_AsAssignment())

In [60]:
import jinja2

basepath = "C:\\Users\\Nathan\\Documents\\Personal\\test\\openapiclient3\\"
template_path = basepath + "templates\\"
out_path = basepath + "out\\"
os.makedirs(out_path, exist_ok=True)


class PythonSwaggerRendererContext():
    model_templates = {
        BasicSchema: "BasicSchema.py.jinja2",
        DateTimeBasicSchema: "BasicSchema.py.jinja2",
        DateBasicSchema: "BasicSchema.py.jinja2",
        ObjectSchema: "ObjectSchema.py.jinja2",
        ArraySchema: "ArraySchema.py.jinja2",
        EnumSchema: "EnumSchema.py.jinja2",
        OneOfSchema: "OneOfSchema.py.jinja2",
#         Response: "Response.py.jinja2"
    }
    
    def __init__(self, path, yctx):
        self.ctx = yctx
        self.loader = jinja2.FileSystemLoader(searchpath=path)
        self.env = jinja2.Environment(loader=self.loader)
        
        self.env.globals.update({
            "repr": repr
        })

    def render_template(self, file, props):
        template = self.env.get_template(file)
        return template.render(**props)

    def load_template(self, file, props):
        try:
            return self.render_template(file, props)
        except jinja2.TemplateSyntaxError as e:
            print("Error loading template '%s'"%file)
            print(e.message)
            print(e.lineno)
            raise Exception(e.message) from None
        
    def get_output_filename(self, ppath, path, file):
        dir = os.path.join(ppath, path)
        dir = os.path.normpath(dir)
        os.makedirs(dir, exist_ok=True)
        if not file.endswith(".py"):
            file += ".py"
        return os.path.join(dir, file)
    
    def render_model(self, obj):
        template = self.model_templates.get(type(obj), None)
        if template is None:
            return "Not implemented for: %s"%(type(obj).__name__)
        return self.load_template(template, self.get_render_env(obj))
    
    def render_response(self, obj):
        # based on code this could be handled by render_model,
        # but I don't know if it would cause problems later
        template = "Response.py.jinja2"
        return self.load_template(template, self.get_render_env(obj))

    def get_render_env(self, obj):
        return {
            "obj": obj, 
            "renderer": self,
            "project": self.current_projet
        }
        
    def handle_file(self, props, ppath):
        file = props['template']
        path = props.get('path', ".")
        out = self.get_output_filename(ppath, path, props['filename'])
#         print("processing %s"%props['filename'])
        result = self.load_template(file, self.get_render_env(props['obj']))
        with open(out, 'w') as f:
            f.write(result)
        
    def handle_project(self, props):
        path = props['base_path']
        project = props['project_name']
        self.current_projet = project
        ppath = os.path.join(path, project)
        ppath = os.path.normpath(ppath)
        for file in props['file_list']:
            self.handle_file(file, ppath)

In [61]:
# class ApiClientProxy():
#     def __init__(self, url, path):
#         self.base_url = url
#         self.server_path = path
        

# helix_url = "https://pbsbiotech.helixalm.cloud:8443"
# helix_path = "/helix-alm/api/v0"
        
# def file_spec(path, template, filename, obj):
#     return {
#         "path": path,
#         "template": template,
#         "filename": filename,
#         "obj": obj
#     }

# project = {
#     "base_path": "C:\\users\\nathan\\documents\\personal\\test\\openapiclient3\\out\\helixalm",
#     "project_name": "helixalmrestapi",
#     "file_list": [
#         file_spec(".", "ApiClient.py.jinja2", "ApiClient", ApiClientProxy(helix_url, helix_path)),
#         file_spec(".\models", "SchemaPrimitives.py.jinja2", "schemabase.py", None)
#     ]
# }

# files = project['file_list']

# model_import_strings = []
# for sch in ctx.schemas.values():        
#     spec = file_spec(".\\models", "ModelSchema.py.jinja2", sch.get_FileName(), sch)
#     files.append(spec)
#     model_import_strings.append("from .%s import %s"%sch.get_AsImport())
# model_spec = file_spec(".\\models", "SubpackageInit.py.jinja2", "__init__.py", model_import_strings)

# for rsp in ctx.responses.values():
#     spec = file_spec(".\\responses", "ResponseTemplateBase.py.jinja2", rsp.get_FileName(), rsp)
#     files.append(spec)

# for cls in ctx.classes.values():
#     spec = file_spec(".\\api", "ApiClassBaseTemplate.py.jinja2", cls.get_FileName(), cls)
#     files.append(spec)

# files.append(model_spec)

# exception_spec = file_spec(".", "Exceptions.py.jinja2", "Exceptions.py", None)
# files.append(exception_spec)

# base_api_spec = file_spec(".", "BaseApiObject.py.jinja2", "BaseApiObject.py", None)
# files.append(base_api_spec)

# def empty_init(path):
#     spec = file_spec(path, "EmptyInit.py.jinja2", "__init__.py", None)
#     files.append(spec)
    
# empty_init(".")
# empty_init(".\\models")
# empty_init(".\\responses")
# empty_init(".\\api")
        
# PythonSwaggerRendererContext(template_path, ctx).handle_project(project)

In [62]:
class ApiClientProxy():
    def __init__(self, url, path):
        self.base_url = url
        self.server_path = path
        

helix_url = "https://pbsbiotech.helixalm.cloud:8443"
helix_path = "/helix-alm/api/v0"
        
def file_spec(path, template, filename, obj):
    return {
        "path": path,
        "template": template,
        "filename": filename,
        "obj": obj
    }

project = {
    "base_path": "C:\\users\\nathan\\documents\\personal\\test\\openapiclient3\\out\\",
    "project_name": "helixalmclient",
    "file_list": [
        file_spec(".", "ApiClient.py.jinja2", "ApiClient", ApiClientProxy(helix_url, helix_path)),
        file_spec(".\models", "SchemaPrimitives.py.jinja2", "schemabase.py", None)
    ]
}

files = project['file_list']

model_import_strings = []
for sch in ctx.schemas.values(): 
#     if isinstance(sch, ArraySchema):
#         continue
    spec = file_spec(".\\models", "ModelSchema.py.jinja2", sch.get_FileName(), sch)
    files.append(spec)
    model_import_strings.append("from .%s import %s"%sch.get_AsImport())
model_spec = file_spec(".\\models", "SubpackageInit.py.jinja2", "__init__.py", model_import_strings)

for rsp in ctx.responses.values():
    spec = file_spec(".\\responses", "ResponseTemplateBase.py.jinja2", rsp.get_FileName(), rsp)
    files.append(spec)

for cls in ctx.classes.values():
    spec = file_spec(".\\api", "ApiClassBaseTemplate.py.jinja2", cls.get_FileName(), cls)
    files.append(spec)

files.append(model_spec)

exception_spec = file_spec(".", "Exceptions.py.jinja2", "Exceptions.py", None)
files.append(exception_spec)

base_api_spec = file_spec(".", "BaseApiObject.py.jinja2", "BaseApiObject.py", None)
files.append(base_api_spec)

def empty_init(path):
    spec = file_spec(path, "EmptyInit.py.jinja2", "__init__.py", None)
    files.append(spec)
    
empty_init(".")
empty_init(".\\models")
empty_init(".\\responses")
empty_init(".\\api")
        
PythonSwaggerRendererContext(template_path, ctx).handle_project(project)
print("done rendering")
import subprocess
import os
here = os.path.abspath(".")
os.chdir(project['base_path'])
try:
    subprocess.check_output('deploy.cmd')
    print("build successful")
finally:
    os.chdir(here)

done rendering
build successful
